In [10]:
import os
import numpy as np
import keras

Parameters

In [11]:
_rotation = 20 
_zoom = 0.5
_flip = True
_batch_size = 128
_random_state = 42
_input_shape = (150, 150, 3)
_dropout = 0.6
_alpha = 1e-5
_epochs = 10
_hidden_unit = 32
_kernel_size = 3
_trainable_layer = 'block5_conv1'

Data preparation

In [12]:
from PIL import Image

def select_image(filename):
    _image = Image.open(filename)
    _image = _image.convert('RGB')
    _image = _image.resize((150,150))
    return np.asarray(_image)

In [13]:
def load_class(directory, label_class, images, labels):
    for filename in os.listdir(directory):
        _path = directory + filename
        images.append(select_image(_path))
        labels.append(label_class)
    return images, labels

In [14]:
def select_dataset(directory):
    _images = list()
    _labels = list()
    for subdir in os.listdir(directory):
        _path = directory + subdir + '/'
        if not os.path.isdir(_path):
            continue
        _images, _labels = load_class(_path, subdir, _images, _labels)
    return _images, _labels

In [16]:
_images, _labels = select_dataset(os.getcwd() + '\\data2\\')
print(len(_images), len(_labels))

190 190


In [17]:
from sklearn.preprocessing import LabelBinarizer

_labelBinarizer = LabelBinarizer()

In [18]:
from keras.utils import to_categorical

def normalize(images, labels):
    images = np.array(images) / 255.0
    labels = np.array(labels)
    labels = _labelBinarizer.fit_transform(labels)
    labels = to_categorical(labels)
    return images, labels

_images, _labels = normalize(_images, _labels)

In [19]:
from sklearn.model_selection import train_test_split

_x_train, _x_test, _y_train, _y_test = train_test_split(_images, _labels, test_size=0.20, stratify=_labels, random_state=_random_state)

In [20]:
from keras.preprocessing.image import ImageDataGenerator

_train_data_augmentation = ImageDataGenerator(rotation_range=_rotation, zoom_range=_zoom, horizontal_flip=_flip)
_train_data_augmentation.fit(_x_train)
_data_augmentation = _train_data_augmentation.flow(_x_train, _y_train, batch_size=_batch_size)

CallBacks

In [21]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

def _get_callbacks(model_name, verbose=1):
    _checkpoint = ModelCheckpoint(os.getcwd() + f'\\models\\{model_name}.hdf5', monitor='val_acc', save_best_only=True, mode='max', verbose=verbose)
    _plateau = ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_delta=_alpha, patience=5, verbose=verbose)
    return [_checkpoint, _plateau]

Model

In [98]:
from datetime import datetime

_epochs = [5, 10, 20]
_models = {
    'Denso': [],
    'LSTM': [LSTM(32, return_sequences=False)],
    'BiLSTM': [Bidirectional(LSTM(32, return_sequences=True)), Bidirectional(LSTM(32, return_sequences=False))],
    'BiGRU': [Bidirectional(GRU(32, return_sequences=False))],
}


for model in _models:
    print('[', datetime.now().strftime("%H:%M:%S"), ']', '=========', model, '=========')
    m = Sequential()
    m.add(Dense(254, activation ='relu', input_shape =_input_shape))
    m.add(Dense(128, activation ='relu'))
    m.add(Dense(64, activation ='relu'))
    m.add(Dropout(0.25))
    m.add(TimeDistributed(Flatten()) if len(_models[model]) > 0 else Flatten())
    for layer in _models[model]:
        m.add(layer)
    m.add(Dense(2, activation ='softmax'))
    m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    for epoch in _epochs:
        print('[', datetime.now().strftime("%H:%M:%S"), ']', 'Quantidade de Épocas:', epoch)
        _callbacks = _get_callbacks(f'{model}_{epoch}', verbose=0)
        m.fit_generator(_data_augmentation,
                        epochs = epoch, 
                        validation_data = (_x_train, _y_train),
                        verbose = 0, 
                        steps_per_epoch=_x_train.shape[0] // _batch_size, 
                        callbacks=_callbacks)
        _result = m.evaluate(_x_train, _y_train, verbose = 0)
        print('[', datetime.now().strftime("%H:%M:%S"), ']', 'Acurácia:', _result[1])
    print('')
        

[ 18:21:45 ] ========= Denso =========
[ 18:21:46 ] Quantidade de Épocas: 5
[ 18:23:19 ] Acurácia: 0.9473684430122375
[ 18:23:19 ] Quantidade de Épocas: 10
[ 18:26:24 ] Acurácia: 1.0
[ 18:26:24 ] Quantidade de Épocas: 20
[ 18:32:24 ] Acurácia: 1.0

[ 18:32:24 ] ========= LSTM =========
[ 18:32:24 ] Quantidade de Épocas: 5
[ 18:34:28 ] Acurácia: 0.7105262875556946
[ 18:34:28 ] Quantidade de Épocas: 10
[ 18:38:22 ] Acurácia: 0.8092105388641357
[ 18:38:22 ] Quantidade de Épocas: 20
[ 18:45:48 ] Acurácia: 0.9078947305679321

[ 18:45:48 ] ========= BiLSTM =========
[ 18:45:50 ] Quantidade de Épocas: 5
[ 18:48:35 ] Acurácia: 0.8026315569877625
[ 18:48:35 ] Quantidade de Épocas: 10
[ 18:53:44 ] Acurácia: 0.9473684430122375
[ 18:53:44 ] Quantidade de Épocas: 20
[ 19:03:38 ] Acurácia: 0.9473684430122375

[ 19:03:38 ] ========= BiGRU =========
[ 19:03:38 ] Quantidade de Épocas: 5
[ 19:06:24 ] Acurácia: 0.5526315569877625
[ 19:06:24 ] Quantidade de Épocas: 10
[ 19:11:08 ] Acurácia: 0.789473712444